This notebook show an example on how to run predictions using a pre-trained a lgsegmentor.

This example it uses the validation data 

In [1]:
import numpy as np
import tifffile
import napari
import leopardgecko.segmentor as lgs

import logging
logging.basicConfig(level=logging.DEBUG)

# Load data  for prediction and respective labels

In [2]:
data_to_pred = tifffile.imread("test_data/TS_0005_crop_val.tif")
labels_to_compare = tifffile.imread("test_data/TS_0005_ribos_membr_crop_val.tif")

In [3]:
print(data_to_pred.shape)

(256, 256, 256)


# Setup lg segmentor

In [4]:
#Create the class
lgsegmentor0 = lgs.cMultiAxisRotationsSegmentor.create_from_model("2023-09-29_ribos_membr.lgsegmod")

DEBUG:root:cMultiAxisRotationsSegmentor __init__() with temp_data_outdir:None , cuda_device:0
DEBUG:root:load_model()


Check default settings and modify as needed

In [5]:
lgsegmentor0.NN1_pred_settings

namespace(quality='high',
          output_probs=True,
          clip_data=True,
          st_dev_factor=2.575,
          data_hdf5_path='/data',
          cuda_device=0,
          downsample=False,
          one_hot=False,
          prediction_axis='Z')

Set CUDA device.

To change cuda device don't change with .cuda_device, but rather use setcudadevice(devnumber)

You may want to check usage and memory-usage with command `!nvidia-smi` to see what is available before starting training

In [6]:
!nvidia-smi

Fri Oct 20 23:20:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:00:05.0 Off |                    0 |
| N/A   63C    P0   206W / 250W |  29897MiB / 32510MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  On   | 00000000:00:06.0 Off |                    0 |
| N/A   

In [7]:
lgsegmentor0.set_cuda_device(1)

Run the prediction and collect result

In [8]:
pred_labels = lgsegmentor0.predict(data_to_pred)

DEBUG:root:predict() data_in.shape:(256, 256, 256), data_in.dtype:int8, use_dask:False
INFO:root:Setting up NN1 prediction
DEBUG:root:clear()
DEBUG:root:NN1_predict()
DEBUG:root:data_to_predict not a list. Converting to list
INFO:root:number of data sets to predict: 1
INFO:root:Data to predict index:0
DEBUG:root:VolSeg2DPredictionManager.__init__()
DEBUG:root:BaseDataManager.__init__()
DEBUG:root:BaseDataManager._preprocess_data()
INFO:root:Calculating mean of data...
INFO:root:Mean value: 3.376452088356018
DEBUG:root:clip_to_uint8()
INFO:root:Calculating standard deviation.
INFO:root:Std dev: 22.28497505498702. Calculating stats.
INFO:root:Lower bound: -54.00735867823556, upper bound: 60.7602628549476
INFO:root:Number of voxels above upper bound to be clipped 95283 - percentage 0.568%
INFO:root:Number of voxels below lower bound to be clipped 175185 - percentage 1.044%
INFO:root:Rescaling intensities.
INFO:root:Data is already in integer dtype, converting to float for rescaling.
INFO:

[########################################] | 100% Completed | 23.77 ss


INFO:root:NN2 prediction complete.


# Visualise result

In [9]:
import napari
NV=napari.Viewer()
NV.add_image(data_to_pred)
NV.add_labels(labels_to_compare)
NV.add_labels(pred_labels, color={1:'green'})

DEBUG:in_n_out:Registering provider of <class 'napari.viewer.Viewer'>: <function _provide_viewer at 0x7f3841ef23a0> (weight: 0, subclassable: True)
DEBUG:in_n_out:Registering provider of <class 'napari.layers.base.base.Layer'>: <function _provide_active_layer at 0x7f3841ef21f0> (weight: 0, subclassable: True)
DEBUG:in_n_out:Registering provider of <class 'napari.components.layerlist.LayerList'>: <function _provide_active_layer_list at 0x7f3841ef2280> (weight: 0, subclassable: True)
DEBUG:in_n_out:Registering processor of <function NewType.<locals>.new_type at 0x7f3841f83550>: <function _add_layer_data_tuples_to_viewer at 0x7f3841ef2430> (weight: 0, subclassable: False)
DEBUG:in_n_out:Registering processor of typing.List[LayerDataTuple]: <function _add_layer_data_tuples_to_viewer at 0x7f3841ef2430> (weight: 0, subclassable: False)
DEBUG:in_n_out:Registering processor of <class 'napari.layers.base.base.Layer'>: <function _add_layer_to_viewer at 0x7f3841f00b80> (weight: 0, subclassable: T

<Labels layer 'pred_labels' at 0x7f383c02b220>

# Compare against ground truth

Use leopardgecko metrics to get voxel-F1 (dice score)

In [10]:
import leopardgecko.metrics as lgm

In [11]:
lgm.MetricScoreOfVols_Dice(pred_labels,labels_to_compare)

INFO:root:MetricScoreOfVols_Dice()
INFO:root:Number of class segmentations in first volume 3
INFO:root:Number of class segmentations in second volume 3
INFO:root:Calculating using numpy
INFO:root:dicescores = {1: 0.5188051691479704, 2: 0.5507384751326172}


(0.5347718221402937, {1: 0.5188051691479704, 2: 0.5507384751326172})

# Run the prediction in blocks and compare with previous result

uses the leopardgecko map_vol_function_by_blocking() function in utils to help with blocking predictions

In [12]:
import leopardgecko.utils as lgu

In [13]:
pred_labels_block = lgu.map_vol_function_by_blocking(lgsegmentor0.predict, data_to_pred, (120,120,120),(20,32,10))

DEBUG:root:map_vol_function_by_blocking() , data3d.shape:(256, 256, 256) ,dtype:int8 block_shape:(120, 120, 120), margins_shape:(20, 32, 10)
DEBUG:root:bl_step:[80, 56, 100]
INFO:root:BLOCK: New block, intended origin iz0,iy0,ix0 = 0,0,0 , use origin iz00,iy00,ix00 = 0,0,0 , end iz1,iy1,ix1 = 120,120,120
INFO:root:BLOCK: Start calculation with this block
DEBUG:root:predict() data_in.shape:(120, 120, 120), data_in.dtype:int8, use_dask:False
INFO:root:Setting up NN1 prediction
DEBUG:root:clear()
DEBUG:root:NN1_predict()
DEBUG:root:data_to_predict not a list. Converting to list
INFO:root:number of data sets to predict: 1
INFO:root:Data to predict index:0
DEBUG:root:VolSeg2DPredictionManager.__init__()
DEBUG:root:BaseDataManager.__init__()
DEBUG:root:BaseDataManager._preprocess_data()
INFO:root:Calculating mean of data...
INFO:root:Mean value: 3.322815972222222
DEBUG:root:clip_to_uint8()
INFO:root:Calculating standard deviation.
INFO:root:Std dev: 21.626499424272946. Calculating stats.
INF

[########################################] | 100% Completed | 2.55 sms


INFO:root:NN2 prediction complete.
INFO:root:BLOCK: This block's calculation completed
INFO:root:BLOCK:Crop block result from origin jz0,jy0,jx0 = : 0,0,0
INFO:root:BLOCK:Copying cropped block to datares
INFO:root:BLOCK: First block result initialises datares
INFO:root:BLOCK: New block, intended origin iz0,iy0,ix0 = 0,0,100 , use origin iz00,iy00,ix00 = 0,0,100 , end iz1,iy1,ix1 = 120,120,220
INFO:root:BLOCK: Start calculation with this block
DEBUG:root:predict() data_in.shape:(120, 120, 120), data_in.dtype:int8, use_dask:False
INFO:root:Setting up NN1 prediction
DEBUG:root:clear()
DEBUG:root:NN1_predict()
DEBUG:root:data_to_predict not a list. Converting to list
INFO:root:number of data sets to predict: 1
INFO:root:Data to predict index:0
DEBUG:root:VolSeg2DPredictionManager.__init__()
DEBUG:root:BaseDataManager.__init__()
DEBUG:root:BaseDataManager._preprocess_data()
INFO:root:Calculating mean of data...
INFO:root:Mean value: 3.382552083333333
DEBUG:root:clip_to_uint8()
INFO:root:Cal

[########################################] | 100% Completed | 2.54 sms


INFO:root:NN2 prediction complete.
INFO:root:BLOCK: This block's calculation completed
INFO:root:BLOCK:Crop block result from origin jz0,jy0,jx0 = : 0,0,10
INFO:root:BLOCK:Copying cropped block to datares
INFO:root:BLOCK: New block, intended origin iz0,iy0,ix0 = 0,0,200 , use origin iz00,iy00,ix00 = 0,0,136 , end iz1,iy1,ix1 = 120,120,256
INFO:root:BLOCK: Start calculation with this block
DEBUG:root:predict() data_in.shape:(120, 120, 120), data_in.dtype:int8, use_dask:False
INFO:root:Setting up NN1 prediction
DEBUG:root:clear()
DEBUG:root:NN1_predict()
DEBUG:root:data_to_predict not a list. Converting to list
INFO:root:number of data sets to predict: 1
INFO:root:Data to predict index:0
DEBUG:root:VolSeg2DPredictionManager.__init__()
DEBUG:root:BaseDataManager.__init__()
DEBUG:root:BaseDataManager._preprocess_data()
INFO:root:Calculating mean of data...
INFO:root:Mean value: 3.346806712962963
DEBUG:root:clip_to_uint8()
INFO:root:Calculating standard deviation.
INFO:root:Std dev: 20.4542

...

INFO:root:NN2 prediction complete.
INFO:root:BLOCK: This block's calculation completed
INFO:root:BLOCK:Crop block result from origin jz0,jy0,jx0 = : 20,32,0
INFO:root:BLOCK:Copying cropped block to datares
INFO:root:BLOCK: New block, intended origin iz0,iy0,ix0 = 240,224,100 , use origin iz00,iy00,ix00 = 136,136,100 , end iz1,iy1,ix1 = 256,256,220
INFO:root:BLOCK: Start calculation with this block
DEBUG:root:predict() data_in.shape:(120, 120, 120), data_in.dtype:int8, use_dask:False
INFO:root:Setting up NN1 prediction
DEBUG:root:clear()
DEBUG:root:NN1_predict()
DEBUG:root:data_to_predict not a list. Converting to list
INFO:root:number of data sets to predict: 1
INFO:root:Data to predict index:0
DEBUG:root:VolSeg2DPredictionManager.__init__()
DEBUG:root:BaseDataManager.__init__()
DEBUG:root:BaseDataManager._preprocess_data()
INFO:root:Calculating mean of data...
INFO:root:Mean value: 3.3200208333333334
DEBUG:root:clip_to_uint8()
INFO:root:Calculating standard deviation.
INFO:root:Std de

[########################################] | 100% Completed | 2.43 sms


INFO:root:NN2 prediction complete.
INFO:root:BLOCK: This block's calculation completed
INFO:root:BLOCK:Crop block result from origin jz0,jy0,jx0 = : 20,32,10
INFO:root:BLOCK:Copying cropped block to datares
INFO:root:BLOCK: New block, intended origin iz0,iy0,ix0 = 240,224,200 , use origin iz00,iy00,ix00 = 136,136,136 , end iz1,iy1,ix1 = 256,256,256
INFO:root:BLOCK: Start calculation with this block
DEBUG:root:predict() data_in.shape:(120, 120, 120), data_in.dtype:int8, use_dask:False
INFO:root:Setting up NN1 prediction
DEBUG:root:clear()
DEBUG:root:NN1_predict()
DEBUG:root:data_to_predict not a list. Converting to list
INFO:root:number of data sets to predict: 1
INFO:root:Data to predict index:0
DEBUG:root:VolSeg2DPredictionManager.__init__()
DEBUG:root:BaseDataManager.__init__()
DEBUG:root:BaseDataManager._preprocess_data()
INFO:root:Calculating mean of data...
INFO:root:Mean value: 3.445274884259259
DEBUG:root:clip_to_uint8()
INFO:root:Calculating standard deviation.
INFO:root:Std de

[########################################] | 100% Completed | 2.64 sms


INFO:root:NN2 prediction complete.
INFO:root:BLOCK: This block's calculation completed
INFO:root:BLOCK:Crop block result from origin jz0,jy0,jx0 = : 20,32,10
INFO:root:BLOCK:Copying cropped block to datares
INFO:root:BLOCK: Completed. Results should be in datares


cleanup temp folders. Don't forget to do this or you will end up with large temp folders

In [14]:
lgsegmentor0.cleanup()

visualise result and compare

In [15]:
import napari
NV=napari.Viewer()
NV.add_image(data_to_pred)
NV.add_labels(labels_to_compare)
NV.add_labels(pred_labels, color={1:'green'})
NV.add_labels(pred_labels_block, color={1:'red'})

DEBUG:in_n_out:Executing @injected ViewerToggleAction.__init__.<locals>.get_current(viewer: napari.viewer.Viewer) with args: (), kwargs: {}
DEBUG:in_n_out:  injecting viewer: <class 'napari.viewer.Viewer'> = Viewer(camera=Camera(center=(0.0, 0.0, 0.0), zoom=1.0, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0, 2, 1), (0, 2, 1)), current_step=(0, 0), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[], help='', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x7f38420c8b80>], _persisted_mouse_event={}, _mouse_drag_gen={}, _mouse_wheel_gen={}, keymap={})
DEBUG:in_n_

<Labels layer 'pred_labels_block' at 0x7f383cb6d580>

DEBUG:napari.loader:ImageSlice.__init__
DEBUG:napari.loader:ImageSlice.__init__
DEBUG:napari.loader:ImageSlice.__init__
DEBUG:napari.loader:ImageSlice.__init__
DEBUG:napari.loader:ImageSlice.__init__
DEBUG:napari.loader:ImageSlice.__init__
DEBUG:napari.loader:ImageSlice.__init__
DEBUG:napari.loader:ImageSlice.__init__
DEBUG:napari.loader:ImageSlice.__init__
DEBUG:napari.loader:ImageSlice.__init__
DEBUG:napari.loader:ImageSlice.__init__
DEBUG:napari.loader:ImageSlice.__init__


Prediction with blocking algorithm is worse from the calculation without blocking so be careful when using it. There are other instances where it may work better

Compare "block"-prediction with ground truth

In [16]:
lgm.MetricScoreOfVols_Dice(pred_labels_block,labels_to_compare)

INFO:root:MetricScoreOfVols_Dice()
INFO:root:Number of class segmentations in first volume 3
INFO:root:Number of class segmentations in second volume 3
INFO:root:Calculating using numpy
INFO:root:dicescores = {1: 0.5114902042615816, 2: 0.4111010845463054}


(0.46129564440394355, {1: 0.5114902042615816, 2: 0.4111010845463054})

In [18]:
lgm.MetricScoreOfVols_Dice(pred_labels,labels_to_compare)

INFO:root:MetricScoreOfVols_Dice()
INFO:root:Number of class segmentations in first volume 3
INFO:root:Number of class segmentations in second volume 3
INFO:root:Calculating using numpy
INFO:root:dicescores = {1: 0.5188051691479704, 2: 0.5507384751326172}


(0.5347718221402937, {1: 0.5188051691479704, 2: 0.5507384751326172})

dice scores, blocking:0.46, non-blocking:0.53, better by non-blocking

Just for curiosity, compare "block"-prediction with whole prediction

In [17]:
lgm.MetricScoreOfVols_Dice(pred_labels_block,pred_labels)

INFO:root:MetricScoreOfVols_Dice()
INFO:root:Number of class segmentations in first volume 3
INFO:root:Number of class segmentations in second volume 3
INFO:root:Calculating using numpy
INFO:root:dicescores = {1: 0.9030428830255378, 2: 0.762003403950374}


(0.8325231434879559, {1: 0.9030428830255378, 2: 0.762003403950374})